<a href="https://colab.research.google.com/github/alirezakavianifar/machineLearning/blob/main/ANN_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import necessary libraries and load the dataset


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import requests
url="https://github.com/2blam/ML/raw/master/deep_learning/Churn_Modelling.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

## Split the data into independant and dependant vatiables and convert categorical variables

In [12]:
X = df.iloc[:, 3:13]
y = df.iloc[:, 13]

geography = pd.get_dummies(X["Geography"],drop_first=True)
gender = pd.get_dummies(X["Gender"], drop_first=True)

In [14]:
X = pd.concat([X,geography,gender], axis=1)

In [15]:
X = X.drop(['Geography','Gender'], axis=1)

## Split the data into training and testing

from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=5)

# Scale the data

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Importing keras libraries

In [27]:
import keras
from keras.models import Sequential
from keras.layers import Dense

## Initializing the ANN

In [28]:
classifier = Sequential()    

## Importing ANN libraries

In [34]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization
from keras.activations import relu, sigmoid

In [30]:

def create_model(layers, activation):
  model = Sequential()
  for i , nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))

    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [31]:
model = KerasClassifier(build_fn=create_model, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [35]:
layers = [[20],[40, 20],[45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size=[128, 256], epochs=[30])
grid=GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

grid_result = grid.fit(X_train, y_train)


## Model best results

In [36]:
print(grid_result.best_score_, grid_result.best_params_)

0.8545000076293945 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [37]:
pred_y = grid.predict(X_test)
y_pred = (pred_y>0.5)

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm=confusion_matrix(y_pred, y_test)

score = accuracy_score(y_pred, y_test)